Link GEE

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=au0OJXFzCv09r2ND87ilYfSGVJDdBVOhBezeoxtz8aU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgvWNz-Ys7c2NztcKyn_difnRuvAY46e0hWCK_6bpFIOhznut7qUHA

Successfully saved authorization token.


Link Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Import dependencies

In [ ]:
!pip install geemap
import geemap as geemap
# import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
# Check shapefile properties
from pprint import pprint
# Tensorflow
import tensorflow as tf
print(tf.__version__)
# Folium
import folium # for visualization
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
print(folium.__version__)

Functions

In [4]:
#@title Mapdisplay: Display GEE objects using folium.
def Mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

1. Data Collection

In [ ]:
# Import shapefiles
center = ee.Geometry.Point([-43.45,-22.93])
RJBox = ee.Geometry.Rectangle([-43.7996, -23.0838, -43.0937, -22.7436])
RJCity = ee.FeatureCollection('users/Flai/2017_RJCity')
subBox = ee.Geometry.Rectangle([-43.4925, -22.9204, -43.4198, -22.8609])
# Get the 2016-1-31 OLI image
image0 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(center).filterDate('2016-01-01', '2016-2-1').sort('CLOUD_COVER').first().clip(RJBox)                
# Use Landsat 8 bands in the prediction: B, G, R, NIR, SWIR1, SWIR2, TIR1, TIR2
band0 = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Plotting variables
# 13-color pallete for raw classified images
Palette13 = [ 'ffffcc', 'ffffcc', 'ffffcc', 'ffffcc', 'ffffcc', 'red', 'ff9933', 'ff9933', 'ff9933', '009933', '8dc33b', '66b3ff', 'red'];
# 6-color palette for combined classified images 
Palette = ['ffffcc', 'red', 'ff9933', '009933', '8dc33b', '66b3ff'];

# Use folium to visualize the imagery.
mapid = image0.getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma':1.4})
map = folium.Map(location=[-22.93,-43.45],zoom_start=10)

folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='False color combination',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

In [6]:
Map = geemap.Map(center=[-22.93,-43.45],zoom=10)
Map
vis_par = {
    'bands': ['B5', 'B4', 'B3'],  
    'min': 0,  
    'max': 4000,  
    'gamma': 1.3
}
#l8vis = {  'bands': ['B5', 'B4', 'B3'],  'min': 0,  'max': 4000,  gamma: 1.3}
Map.addLayer(image0,vis_par,'image0')

Training and Testing samples

In [7]:
train0 = ee.FeatureCollection('users/Flai/MCS/train200209')
test = ee.FeatureCollection('users/Flai/MCS/test_200205_singlepoint')
print(test.getInfo())
# Print the number
print('Count: ',test.size().getInfo())
# Map the test samples
centroid = test.geometry().centroid().getInfo()['coordinates']
centroid.reverse()
#Mapdisplay(centroid,{'test':test.getInfo()},zoom_start=10)

{'type': 'FeatureCollection', 'columns': {'Label': 'Integer', 'ORIG_FID': 'Long', 'ROI_NAME': 'String', 'ROI_NPTS': 'Float', 'system:index': 'String'}, 'version': 1597953854210303, 'id': 'users/Flai/MCS/test_200205_singlepoint', 'properties': {'system:asset_size': 37169}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-43.31557831924271, -22.806955239857523]}, 'id': '00000000000000000000', 'properties': {'Label': 1, 'ORIG_FID': 0, 'ROI_NAME': '1', 'ROI_NPTS': 160}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-43.36478449936749, -22.81422357457464]}, 'id': '00000000000000000001', 'properties': {'Label': 1, 'ORIG_FID': 0, 'ROI_NAME': '1', 'ROI_NPTS': 160}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-43.295334000460066, -22.80827513376935]}, 'id': '00000000000000000002', 'properties': {'Label': 1, 'ORIG_FID': 0, 'ROI_NAME': '1', 'ROI_NPTS': 160}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates':

Base Classifier Results

In [10]:
MLP6 = ee.Image('users/Flai/MCS/1_MLP_0209').select(['b1'],['classification'])
MLC6 = ee.Image('users/Flai/MCS/2_MLC_0209').select(['b1'],['classification'])
MD6  = ee.Image('users/Flai/MCS/3_MD_0820')
SVM6 = ee.Image('users/Flai/MCS/4_SVM_0209').select(['b1'],['classification'])

MLP=MLP6
MLC=MLC6
MD=MD6
SVM = SVM6
# Mask out 0s
#mask = MLP6.And(SVM6).And(MD6).And(MLC6);
#MLP = MLP6.updateMask(mask);
#SVM = SVM6.updateMask(mask);
#MD = MD6.updateMask(mask);
#MLC = MLC6.updateMask(mask);

Base Classifier Evaluation

In [11]:
# Test sample labels
y_true = test.aggregate_array('Label').getInfo() 
# Base classification outputs
MLP_pred = MLP.reduceRegions(test, ee.Reducer.first()).aggregate_array('first').getInfo()
MLC_pred = MLC.reduceRegions(test, ee.Reducer.first()).aggregate_array('first').getInfo()
MD_pred = MD.reduceRegions(test, ee.Reducer.first()).aggregate_array('first').getInfo()
SVM_pred = SVM.reduceRegions(test, ee.Reducer.first()).aggregate_array('first').getInfo()
"""
from sklearn.metrics import classification_report
class_names = ['Formal','Informal','Barren','Forest','Grassland','Waters']
print('MLP report',classification_report(y_true, MLP_pred, target_names=class_names))
print('MLC report',classification_report(y_true, MLC_pred, target_names=class_names))
print('MD report',classification_report(y_true, MD_pred, target_names=class_names))
print('SVM report',classification_report(y_true, SVM_pred, target_names=class_names))
"""
# Overall Accuracy
from sklearn.metrics import accuracy_score
MLP_OA = accuracy_score(y_true, MLP_pred); MLC_OA = accuracy_score(y_true, MLC_pred)
MD_OA = accuracy_score(y_true, MD_pred);   SVM_OA = accuracy_score(y_true, SVM_pred)
print(MLP_OA, MLC_OA, MD_OA, SVM_OA)

# F1 Score
from sklearn.metrics import f1_score
MLP_f = f1_score(y_true, MLP_pred, average=None); MLC_f = f1_score(y_true, MLC_pred, average=None)
MD_f  = f1_score(y_true, MD_pred, average=None);  SVM_f = f1_score(y_true, SVM_pred, average=None)
print(MLP_f, MLC_f, MD_f, SVM_f)
A_SVM= confusion_matrix(SVM_pred, y_true)

# Precision Score
from sklearn.metrics import precision_score
MLP_p = precision_score(y_true, MLP_pred, average=None); MLC_p = precision_score(y_true, MLC_pred, average=None)
MD_p  = precision_score(y_true, MD_pred, average=None);  SVM_p = precision_score(y_true, SVM_pred, average=None)
print(MLP_p, MLC_p, MD_p, SVM_p)

# Recall Score
from sklearn.metrics import recall_score
MLP_r = recall_score(y_true, MLP_pred, average=None); MLC_r = recall_score(y_true, MLC_pred, average=None)
MD_r  = recall_score(y_true, MD_pred, average=None);  SVM_r = recall_score(y_true, SVM_pred, average=None)
print(MLP_r, MLC_r, MD_r, SVM_r)

0.7189655172413794 0.7189655172413794 0.75 0.7931034482758621
[0.57718121 0.56108597 0.56470588 0.90697674 0.8159204  0.94845361] [0.44262295 0.55290102 0.88       0.91402715 0.9009901  0.92      ] [0.68765133 0.56626506 0.79120879 0.87323944 0.84210526 0.85057471] [0.73125    0.63461538 0.70588235 0.921875   0.84375    0.96969697]
[0.62318841 0.51239669 0.68571429 0.84782609 0.81188119 0.9787234 ] [0.64285714 0.41968912 0.88       1.         0.89215686 0.92      ] [0.56126482 0.71212121 0.87804878 1.         0.88888889 1.        ] [0.73125    0.61111111 0.85714286 0.86764706 0.88043478 0.97959184]
[0.5375 0.62   0.48   0.975  0.82   0.92  ] [0.3375     0.81       0.88       0.84166667 0.91       0.92      ] [0.8875 0.47   0.72   0.775  0.8    0.74  ] [0.73125    0.66       0.6        0.98333333 0.81       0.96      ]


Secondary Input Compilation

In [ ]:
# Class Accuracy layer (F-score)
def CAlayer (base, a): #image and accuracy report
  return base.remap([1,2,3,4,5,6],[a[0], a[1], a[2], a[3], a[4], a[5]])
MLP_fl = CAlayer(MLP, MLP_f); MLC_fl = CAlayer(MLC, MLC_f)
MD_fl = CAlayer(MD, MD_f);    SVM_fl = CAlayer(SVM, SVM_f)
MLP_pl = CAlayer(MLP, MLP_p); MLC_pl = CAlayer(MLC, MLC_p)
MD_pl = CAlayer(MD, MD_p);    SVM_pl = CAlayer(SVM, SVM_p)
MLP_rl = CAlayer(MLP, MLP_r); MLC_rl = CAlayer(MLC, MLC_r)
MD_rl = CAlayer(MD, MD_r);    SVM_rl = CAlayer(SVM, SVM_r)
# Combine Base Classification Results with F-score layer
image1 = ee.Image.cat([MLP, MLC, MD, SVM, MLP_fl, MLC_fl, MD_fl, SVM_fl])
# image1.getInfo()
band1 = ['remapped','remapped_1','remapped_2','remapped_3'];
band2 = ['classification', 'classification_1', 'classification_2', 'classification_3'];
band3 = ['classification', 'classification_1', 'classification_2', 'classification_3', 'remapped','remapped_1', 'remapped_2', 'remapped_3'];
# Use folium to visualize the imagery.
mapid = image1.select('remapped_2').getMapId({'min': 0, 'max': 1, 'gamma':1.4})
map = folium.Map(location=[-22.93,-43.45],zoom_start=10)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='MDF1',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Subset training samples

In [ ]:
# Convert training label to a raster to add as a band to the classification result
labeltr = train0.reduceToImage(['No6'], ee.Reducer.first())
# Count correct classification 
MLP_m = MLP.eq(labeltr); SVM_m = SVM.eq(labeltr);
MD_m = MD.eq(labeltr);  MLC_m = MLC.eq(labeltr); 
# Sum up correct classification among base classifiers
sum = MLP_m.add(MLC_m).add(MD_m).add(SVM_m);

# Function: sampling a subtrainset with >=N correct base outputs from an input image 
def train_fil(N): 
  """
  Create a mask where at least N classifiers output correctly
  """
  filter = sum.updateMask(sum.gte(N)).addBands(sum);
  # Convert the mask to vector
  bound = filter.reduceToVectors(geometryType='polygon', reducer=ee.Reducer.first());
  # Filter the training sets with the vector mask
  subtrain = train0.filterBounds(bound);
  # Apply trainset to the input image
  return subtrain
  #sampling = InputImg.sampleRegions(collection=subtrain, properties=['label'], scale=30);
  # Print(subtrain.getInfo());
subtrain1 = train_fil(1); subtrain2 = train_fil(2); 
subtrain3 = train_fil(3); subtrain4 = train_fil(4); 

(Back up) Inversed training sample filtering mechanism




In [ ]:
# Backup - Function: sampling a subtrainset with <N correct base outputs from an input image 
def train_fil(N): 
  """
  Create a mask where at least N classifiers output correctly
  """
  filter = sum.updateMask(sum.lt(N)).updateMask(sum.gte(2)).addBands(sum);
  # Convert the mask to vector
  bound = filter.reduceToVectors(geometryType='polygon', reducer=ee.Reducer.first());
  # Filter the training sets with the vector mask
  subtrain = train0.filterBounds(bound);
  # Apply trainset to the input image
  return subtrain
  #sampling = InputImg.sampleRegions(collection=subtrain, properties=['label'], scale=30);
  # Print(subtrain.getInfo());
#subtrain1 = train_fil(1); subtrain2 = train_fil(2); 
#subtrain3 = train_fil(3); subtrain4 = train_fil(4); 
subtrain5 = train_fil(4); 

Export subtrain2 to drive

In [ ]:
#geemap.ee_export_image_to_drive(MLP_fl, description= 'MLP_F1', folder='Colabs',scale=30, region=RJBox)
geemap.ee_export_vector_to_drive(subtrain2, description= 'subtrain2', folder='Colabs')

Transform data into Array

In [12]:
def getList(Img, sample): 
  """Get a list of sample value from a single-band image"""
  return Img.reduceRegions(sample, ee.Reducer.first()).aggregate_array('first').getInfo()

def getArray_map(sample): 
  """Get an array of sample values in 4 output layers"""
  MLP_b1=getList(MLP6, sample); MLC_b1 = getList(MLC6, sample)
  MD_b1 =getList(MD6, sample); SVM_b1 = getList(SVM6, sample)
  # print(len(MLC_b2))
  InputArray = np.column_stack((MLP_b1, MLC_b1, MD_b1, SVM_b1))
  return InputArray

def getArray_f1(sample): 
  """Get an array of sample values in f1 layers"""
  MLP_b=getList(MLP_fl, sample); MLC_b = getList(MLC_fl, sample)
  MD_b =getList(MD_fl, sample); SVM_b = getList(SVM_fl, sample)
  InputArray = np.column_stack((MLP_b, MLC_b, MD_b, SVM_b))
  return InputArray

def getArray_precision(sample): 
  """Get an array of sample values in precision layers"""
  MLP_b=getList(MLP_pl, sample); MLC_b = getList(MLC_pl, sample)
  MD_b =getList(MD_pl, sample); SVM_b = getList(SVM_pl, sample)
  InputArray = np.column_stack((MLP_b, MLC_b, MD_b, SVM_b))
  return InputArray

def getArray_recall(sample): 
  """Get an array of sample values in recall layers"""
  MLP_b=getList(MLP_rl, sample); MLC_b = getList(MLC_rl, sample)
  MD_b =getList(MD_rl, sample); SVM_b = getList(SVM_rl, sample)
  InputArray = np.column_stack((MLP_b, MLC_b, MD_b, SVM_b))
  return InputArray

MCS classification and evaluation function

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

# Define the randome forest model
rf = RandomForestClassifier(n_estimators=30, criterion='gini',min_samples_split=3,random_state=0)

def MCS(trainset, train_input, test_input):
  target = trainset.aggregate_array('No6').getInfo()
  # Fit our model to training data
  Model = rf.fit(train_input, target)
  # Now predict for the test pixel
  y_pred = rf.predict(test_input)
  # print(test_input.shape)
  print(np.unique(y_pred))
  # Accuracy assessment
  class_names = ['Formal','Informal','Barren','Forest','Grassland','Waters'] 
  '''
  change the things you want to output HERE
  '''
  print(accuracy_score(y_true, y_pred),f1_score(y_true, y_pred, average=None))
  # print(np.unique(y_pred))
  #classification_report(y_true, y_pred, target_names=class_names))
  #A = confusion_matrix(y_pred, y_true)
  #B = cohen_kappa_score(y_pred, y_true)
  #return A, B 
  '''
  # Feature Importance
  bands = ['MLP', 'MLC', 'MD', 'SVM', 'MLP_fl', 'MLC_fl','MD_fl',"SVM_fl"]
  for b, imp in zip(bands, rf.feature_importances_):
    print('Layer {b} importance: {imp}'.format(b=b, imp=imp))
  '''

Original Trainset

In [ ]:
def InpExp(trainset):
  # 4 classification bands
  X1=getArray_map(trainset); X1_test=getArray_map(test); #print(X1_test.shape)
  print('Input: 4 classification bands:'); MCS(trainset, X1, X1_test)
  # 4 F1-score bands
  X2=getArray_f1(trainset); X2_test=getArray_f1(test)
  print('Input: 4 F1-score bands:'); MCS(trainset, X2, X2_test)
  # 4 Precision bands
  X3 = getArray_precision(trainset); X3_test=getArray_precision(test)
  print('Input: 4 Precision bands:'); MCS(trainset, X3, X3_test)
  # 4 Recall bands
  X4 = getArray_recall(trainset); X4_test=getArray_recall(test)
  print('Input: 4 Recall bands:'); MCS(trainset, X4, X4_test)
print('Original training samples results:\n')
InpExp(train0)

Original training samples results:

Input: 4 classification bands:
[1 2 3 4 5 6]
0.7931034482758621 [0.79268293 0.6        0.86597938 0.87323944 0.84656085 0.89320388]
Input: 4 F1-score bands:
[1 2 3 4 5 6]
0.7982758620689655 [0.79878049 0.61061947 0.85714286 0.87323944 0.87912088 0.84955752]
Input: 4 Precision bands:
[1 2 3 4 5 6]
0.7948275862068965 [0.79510703 0.61061947 0.86       0.86792453 0.86486486 0.85454545]
Input: 4 Recall bands:
[1 2 3 4 5 6]
0.8258620689655173 [0.79878049 0.67647059 0.84848485 0.90265487 0.8952381  0.88172043]


N=1

In [ ]:
print('Filtering condition: N=1 results:\n')
InpExp(subtrain1)

Filtering condition: N=1 results:

Input: 4 classification bands:
[1 2 4 6]
0.1310344827586207 [0.23859649 0.26056338 0.         0.         0.         0.03597122]
Input: 4 F1-score bands:
[1 2 4 6]
0.17586206896551723 [0.3785489  0.25482625 0.         0.         0.         0.06569343]
Input: 4 Precision bands:
[1 2 4 6]
0.16206896551724137 [0.39721254 0.26506024 0.         0.         0.         0.02597403]
Input: 4 Recall bands:
[1 2 4 6]
0.16896551724137931 [0.4        0.19760479 0.         0.         0.         0.06278027]


N=2

In [ ]:
print('Filtering condition: N=2 results:\n')
InpExp(subtrain2)

Filtering condition: N=2 results:

Input: 4 classification bands:
[1 2 3 4 6]
0.1879310344827586 [0.27838828 0.28169014 0.68181818 0.         0.         0.00775194]
Input: 4 F1-score bands:
[1 2 3 4 5 6]
0.16551724137931034 [0.26819923 0.27681661 0.33802817 0.02515723 0.07692308 0.02173913]
Input: 4 Precision bands:
[1 2 3 4 5 6]
0.15 [0.14035088 0.32911392 0.16883117 0.         0.07692308 0.0199005 ]
Input: 4 Recall bands:
[1 2 3 4 6]
0.15862068965517243 [0.24       0.2320442  0.22429907 0.07092199 0.         0.03      ]


N=3

In [ ]:
print('Filtering condition: N=3 results:\n')
InpExp(subtrain3)

Filtering condition: N=3 results:

Input: 4 classification bands:
[1 2 3 4 5 6]
0.5465517241379311 [0.44078947 0.31147541 0.79569892 0.73387097 0.81283422 0.19047619]
Input: 4 F1-score bands:
[1 2 3 4 5 6]
0.5517241379310345 [0.44       0.30769231 0.80851064 0.736      0.82795699 0.21686747]
Input: 4 Precision bands:
[1 2 3 4 5 6]
0.5448275862068965 [0.42567568 0.30645161 0.8125     0.73895582 0.8172043  0.18823529]
Input: 4 Recall bands:
[1 2 3 4 5 6]
0.5224137931034483 [0.44816054 0.32478632 0.35114504 0.86792453 0.8040201  0.07058824]


N=4

In [ ]:
print('Filtering condition: N=4 results:\n')
InpExp(subtrain4)

Filtering condition: N=4 results:

Input: 4 classification bands:
[1 2 3 4 5 6]
0.7637931034482759 [0.74193548 0.61276596 0.83870968 0.86792453 0.84210526 0.75      ]
Input: 4 F1-score bands:
[1 2 3 4 5 6]
0.7689655172413793 [0.73442623 0.6        0.84536082 0.87850467 0.85561497 0.8034188 ]
Input: 4 Precision bands:
[1 2 3 4 5 6]
0.7034482758620689 [0.73026316 0.60251046 0.85148515 0.73895582 0.85106383 0.25316456]
Input: 4 Recall bands:
[1 2 3 4 5 6]
0.7655172413793103 [0.73026316 0.61016949 0.84536082 0.86792453 0.82352941 0.82242991]


2<=N<4

In [ ]:
print('Filtering condition: N=1 results:\n')
InpExp(subtrain5)

Filtering condition: N=1 results:

Input: 4 classification bands:
[1 2 3 4 5 6]
0.8051724137931034 [0.80357143 0.68341709 0.88       0.82511211 0.80769231 0.93617021]
Input: 4 F1-score bands:
[1 2 3 4 5 6]
0.8103448275862069 [0.80924855 0.71657754 0.703125   0.86792453 0.84656085 0.93877551]
Input: 4 Precision bands:
[1 2 3 4 5 6]
0.8137931034482758 [0.80346821 0.71957672 0.81081081 0.86792453 0.80769231 0.93617021]
Input: 4 Recall bands:
[1 2 3 4 5 6]
0.8155172413793104 [0.80813953 0.72340426 0.88888889 0.86792453 0.80382775 0.85185185]


Z stat Function

In [ ]:
def Kvar(A):
  '''
  Calculate the kappa variance from a confusion matrix "A"
  '''
  N=len(A[0]) # Get the number of classes
  # Add sum of rows and cols to the matrix
  col_sum = np.sum(A,0)
  A = np.r_[A,[col_sum]] # Add a sum row to the bottom
  row_sum = np.sum(A,1) # Add a sum col to the right
  A = np.c_[A,row_sum]
  n = A[N,N] # The total number of testing samples

  sum_ii=0; sum_ij=0; sum_c=0; sum_d=0;
  for i in range(N):
    sum_ii=sum_ii+A[i,i]
    sum_ij=sum_ij+A[i,N]*A[N,i]
    sum_c=sum_c + A[i,i]*(A[i,N]+A[N,i])
    temp=0
    for j in range(N):
      temp = temp + A[i,j]*(A[j,N]+A[N,i])**2
    sum_d = sum_d + temp
  t1=1/n*sum_ii
  t2=1/(n**2)*sum_ij
  t3=1/(n**2)*sum_c
  t4=1/(n**3)*sum_d
  
  a1=t1*(1-t1)/(1-t2)**2
  a2=2*(1-t1)*(2*t1*t2-t3)/(1-t3)**3
  a3=(1-t1)**2*(t4-4*t2**2)/(1-t2)**4
  v=1/n*(a1+a2+a3)
  return v

def Zstat(v,kappa):
  '''
  v: A list of kappa variance
  kappa: A list of cohen kappa
  '''
  # Calculate Z statistics
  a = kappa/np.sqrt(v)
  # Pair-wise comparison
  N = len(v)  # number of classifier being compared
  b = np.zeros([N,N])
  for i in range(N):
    b[i]=(np.array(kappa) - np.array(kappa)[i])/np.sqrt(np.array(v)+np.array(v)[i])
  b = np.r_[b, [a]]
  print(b)
  return b

Get the confusion matrix and kappa index

In [ ]:
# Compare between training subsets (using F1 score)
X0=getArray_f1(train0) 
X1=getArray_f1(subtrain1)
X2=getArray_f1(subtrain2) 
X3=getArray_f1(subtrain3)
X4=getArray_f1(subtrain4)
f_test=getArray_f1(test)

Af0, K_f0 =  MCS(train0, X0, f_test)
Af1, K_f1 =  MCS(subtrain1, X1, f_test)
Af2, K_f2 =  MCS(subtrain2, X2, f_test)
Af3, K_f3 =  MCS(subtrain3, X3, f_test)
Af4, K_f4 =  MCS(subtrain4, X4, f_test)

# Compare between training subsets (using F1 score)
v1 = [Kvar(Af0),Kvar(Af1),Kvar(Af2),Kvar(Af3),Kvar(Af4)]
kappa1 = [K_f0,K_f1,K_f2,K_f3,K_f4]
Zstat(v1,kappa1)

In [ ]:
# Compare between input (N=2)
map = getArray_map(subtrain2)
map_test=getArray_map(test)
f1 = X2
f1_test=f_test
pc = getArray_precision(subtrain2)
pc_test = getArray_precision(test)
rc = getArray_recall(subtrain2)
rc_test = getArray_recall(test)

In [ ]:
Am0, K_m0 = MCS(subtrain2, map, map_test)
Af2, K_f2 = MCS(subtrain2, f1, f1_test)
Apc, K_pc = MCS(subtrain2, pc, pc_test)
Arc, K_rc = MCS(subtrain2, rc, rc_test)
# Compare between input (N=0)
v2 = [Kvar(Am0),Kvar(Af2),Kvar(Apc),Kvar(Arc)]
kappa2 = [K_m0,K_f2, K_pc, K_rc]
Zstat(v2,kappa2)

Export the image band by band

In [ ]:
geemap.ee_export_image_to_drive(MLP_fl, description= 'MLP_F1', folder='Colabs',scale=30, region=RJBox)
geemap.ee_export_image_to_drive(MLC_fl, description= 'MLC_F1', folder='Colabs',scale=30, region=RJBox)
geemap.ee_export_image_to_drive(MD_fl, description=   'MD_F1', folder='Colabs',scale=30, region=RJBox)
geemap.ee_export_image_to_drive(SVM_fl, description= 'SVM_F1', folder='Colabs',scale=30, region=RJBox)

Read the F1-score image band by band

In [ ]:
#!pip install rasterio
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image
def band_to_array (data_path):
  '''
  import single band image into an array
  '''
  with rasterio.open(data_path, 'r') as ds:
    data = ds.read()
    data = reshape_as_image(data) # the rasterio.read output is originally (D, W, H), make it (W, H, D) here
    #data[data == -9999] = np.nan
    data[np.isneginf(data)] = 0 #np.nan
    data[np.isnan(data)] = 0
    data[np.isinf(data)]=0
    print(data.shape)
    data_vector = data.reshape([data.shape[0] * data.shape[1], data.shape[2]])
    print(data_vector.shape)
  return data_vector
# Read the four F1-score layers
p1 = '/content/drive/My Drive/Colabs/MLP_F1.tif'; p2 = '/content/drive/My Drive/Colabs/MLC_F1.tif'
p3 = '/content/drive/My Drive/Colabs/MD_F1.tif'; p4 = '/content/drive/My Drive/Colabs/SVM_F1.tif'
a1 = band_to_array(p1); a2 = band_to_array(p2);
a3 = band_to_array(p3); a4 = band_to_array(p4);
mcs_inp = np.concatenate((a1,a2,a3,a4),axis=1)
mcs_inp.shape

Prediction over a whole image

In [ ]:
train_input = getArray_f1(train0)
target = train0.aggregate_array('No6').getInfo()
# Fit our model to training data
Model = rf.fit(train_input, target)
# Now predict for the test pixel
pred_map = rf.predict(mcs_inp)
print(np.unique(pred_map))
# Change back to 2D
Z = pred_map.reshape([1282, 2428]).astype('int32')    

[1 2 3 4 5 6]


Display the image

In [ ]:
from matplotlib import pyplot
from matplotlib import colors
# Color Assignment
cmap = colors.ListedColormap(['gray', 'red','orange','green', 'green', 'blue'])
#cmap0 = colors.ListedColormap(['ffffcc', 'red', 'ff9933', '009933', '8dc33b', '66b3ff'])
pyplot.figure(figsize = (20,10))
pyplot.imshow(Z, cmap=cmap)
pyplot.show() 

Write the prediction to a tiff

In [ ]:
# Open an georeferenced image as the framework to hold the output values. 
frame = rasterio.open(p1, 'r')
# Write the prediction values into a new image with the frame
with rasterio.open(
    '/content/drive/My Drive/Colabs/Test0.tif',
    'w',
    driver='GTiff',
    height=frame.height,
    width=frame.width,
    count=1,
    dtype=Z.dtype,
    crs=frame.crs,
    transform = frame.transform,
) as dst:
    dst.write(Z, 1)
frame.close()

Rasterio Experiments

In [ ]:
dataset = rasterio.open('/content/drive/My Drive/Colabs/SecImg_sub.tif')
# Dimensions
dataset.width
dataset.height
dataset.shape
# Band information
dataset.indexes
# Read the layer 1 DN values
dataset.read(1)[dataset.height // 2, dataset.width // 2] # "//" stands for integer division
# Display the image
from matplotlib import pyplot
pyplot.imshow(dataset.read(1), cmap='gray')
pyplot.show()  
# Read all bands of the input dataset into a 3-dimensonal ndarray
array = dataset.read() # the 3 axes are (bands, rows, columns)
from rasterio.plot import reshape_as_raster, reshape_as_image
array = reshape_as_image(array) # the 3 axes is (rows, columns, bands)
# Find out Null values
for i, dtype, nodataval in zip(dataset.indexes, dataset.dtypes, dataset.nodatavals):
    print(i, dtype, nodataval)
# Close the dataset
dataset.close()